In [34]:
import numpy as np
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# importing the required modules
import glob
import pandas as pd

cols = ['title','subTitle','imageUrls','categoryName','description','address','city','state','neighborhood',
   'reviewsTags', 'totalScore']

# cols1 = ['title','subTitle','categoryName','description','address','city','state','neighborhood',
#    'reviewsTags', 'totalScore']

# specifying the path to csv files
path = "/content/drive/MyDrive/IR Project Dataset/state-wise Dataset"

# csv files in the path
file_list = glob.glob(path + "/*.xlsx")

# list of excel files we want to merge.
# pd.read_excel(file_path) reads the
# excel data into pandas dataframe.
excl_list = []
images_df = []
for file in file_list:
  print(file)
  data_df = pd.read_excel(file)
  #----------------filling all missing values of 'state' feature -----------------
  res = data_df[data_df['state'].notnull()]
  state_value_to_fill = res['state'].iloc[0] 
  data_df['state'] = data_df['state'].fillna(state_value_to_fill)
  #----------------------------------------x------------------------------------

#----------------filling all missing values of 'city' feature -----------------
  res = data_df[data_df['city'].notnull()]
  state_value_to_fill = res['state'].iloc[0] 
  data_df['city'] = data_df['city'].fillna(state_value_to_fill)
#----------------------------------------x------------------------------------

  #---------------Replacing with subTitle value if any entry has nan in title feature----------
  data_df['title'] = data_df['title'].mask(data_df['title'].isna(), data_df['subTitle'])
#---------------------------------------------------------x-----------------------------------



  #---------------Replacing with 'peopleSearchFor' value if any entry has nan in title feature----------
  count = 0
  people_search_for = ''
  for i in data_df.columns:
    if(i == 'peopleAlsoSearch'):
      count = 2
    elif(count == 2):
      people_search_for = i
      count = 0
  data_df['title'] = data_df['title'].mask(data_df['title'].isna(), data_df[people_search_for])
#---------------------------------------------------------x-----------------------------------



#------------------------- Remove all entries that has nan in title feature ----------------------
  dele_rows = list(data_df[data_df['title'].isnull()].index)
  data_df = data_df.drop(dele_rows)
#------------------------------------------x--------------------------------------

#-------------------------Remove all enteries that has nan in categoryName feature
  dele_rows = list(data_df[data_df['categoryName'].isnull()].index)
  data_df = data_df.drop(dele_rows)
#------------------------------------------------x------------------------------


  #---------------Replacing with neighborhood value if any entry has nan in address feature----------
  data_df['address'] = data_df['address'].mask(data_df['address'].isna(), data_df['neighborhood'])
#----------------------------------------------x-----------------------------


#------------------------- Replacing remaining entries that has nan in address feature with city/state name ----------------------
  data_df['address'] = data_df['address'].mask(data_df['address'].isna(), data_df['city'])
  data_df['address'] = data_df['address'].mask(data_df['address'].isna(), data_df['state'])
  #------------------------------------------x--------------------------------------

#---------------Replacing with neighborhood value if any entry has nan in address feature----------
  data_df['neighborhood'] = data_df['neighborhood'].mask(data_df['neighborhood'].isna(), data_df['address'])
#----------------------------------------------x-----------------------------

  #------------------------- Replacing  entries that has nan in city feature with state name ----------------------
  data_df['city'] = data_df['city'].mask(data_df['city'].isna(), data_df['state'])
  #------------------------------------------x--------------------------------------

  #------------------------- Replacing  entries that has nan in description feature ----------------------
  data_df['description'] = data_df['description'].mask(data_df['description'].isna(), 'no information available')
  #------------------------------------------x--------------------------------------


    #------------------------- Replacing  entries that has nan in reviewTags feature with categoryName name ----------------------
  data_df['reviewsTags'] = data_df['reviewsTags'].mask(data_df['reviewsTags'].isna(), data_df['categoryName'])
  #------------------------------------------x--------------------------------------

    #------------------------- Replacing  entries that has nan in totalScore feature ----------------------
  data_df['totalScore'].fillna(data_df['totalScore'].median(), inplace=True)
  #------------------------------------------x--------------------------------------



  excl_list.append(data_df)
  
  #excl_list.append(pd.read_excel(file,usecols = cols))



# concatenate all DataFrames in the list
# into a single DataFrame, returns new
# DataFrame.
excl_merged = pd.concat(excl_list, ignore_index=True)

# exports the dataframe into excel file
# with specified name.
excl_merged.to_excel('state_wise_data.xlsx', index=False)


/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/maharashtra.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/delhi.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/himachalpradesh.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/uttarakhand.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/kerala.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/west bengal.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/telangana.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/tamil nadu.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/haryana.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/jammu and kashmir.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/Andaman and Nicobar Islands.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/Assam.xlsx
/content/drive/MyDrive/IR Project Dataset/state-wise Dataset/C

In [ ]:
# response = requests.get('https://lh5.googleusercontent.com/p/AF1QipN-94nW345oInCxyLvsmPFO_0MxS_K9jileT09Q=w1920-h1080-k-no')
# image_io = BytesIO(response.content)

# img = Image.open(image_io)
# img = img.resize((150, 150)) # resize to 150x150x3
# img = np.array(img)

In [66]:
travel_data = pd.read_excel('final_full_state_wise_data.xlsx')

In [67]:
travel_data['imageUrls'].isnull().sum()

515

In [44]:
travel_data[travel_data['imageUrls'].isnull()].index

Int64Index([ 214,  215,  218,  219,  220,  339,  434,  485,  744,  750,
            ...
            3221, 3230, 3231, 3235, 3237, 3242, 3243, 3245, 3246, 3259],
           dtype='int64', length=515)

In [68]:
missing_images_index = list(travel_data[travel_data['imageUrls'].isnull()].index)

In [70]:
len(missing_images_index)

515

In [ ]:
import requests
from bs4 import BeautifulSoup
for i in missing_images_index:
  word = travel_data['title'][i]
  if(pd.isnull(travel_data['imageUrls'][i])):
    
    url = 'https://www.google.com/search?q={0}&tbm=isch'.format(word)
    content = requests.get(url).content
    soup = BeautifulSoup(content,'lxml')
    images = soup.findAll('img')
    for image in images:
      print(image.get('src'))
      #travel_data['imageUrls'][i] = image.get('src')
      break

In [73]:
pip install bing-image-urls

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 8.8 MB/s eta 0:00:00


In [76]:
from bing_image_urls import bing_image_urls
for i in missing_images_index:
  word = travel_data['title'][i]
  if(pd.isnull(travel_data['imageUrls'][i])):
    url = bing_image_urls(word, limit=1)[0]
    travel_data['imageUrls'][i] = url

<ipython-input-76-d0d8878a7271>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  travel_data['imageUrls'][i] = url


In [77]:
travel_data['imageUrls'].isnull().sum()

0

In [78]:
for i in missing_images_index:
  print(travel_data['imageUrls'][i])

https://janmatsamachar.com/wp-content/uploads/2020/07/images.jpeg-113.jpg
http://www.pxleyes.com/images/contests/holy-moly-2/fullsize/Viewpoint-4ff5ca8bc4519_hires.jpg
https://graynson.com/wp-content/uploads/2021/04/site-construction-scaled.jpeg
https://lp-cms-production.imgix.net/2019-06/GettyImages-149353949_high.jpg?fit=crop&amp;q=40&amp;sharp=10&amp;vib=20&amp;auto=format&amp;ixlib=react-8.6.4
https://www.holidify.com/images/bgImages/HIMACHAL-PRADESH.jpg
https://www.tourmyindia.com/socialimg/uttarakhand-tourism.jpg
http://www.beautifulpacific.com/fiji-holidays/pictures/Kadavu-01.jpg
https://s.yimg.com/uu/api/res/1.2/9B12O3rmbGd4rlWTw_sjIA--~B/aD0zMzMzO3c9NTAwMDtzbT0xO2FwcGlkPXl0YWNoeW9u/https://media-mbst-pub-ue1.s3.amazonaws.com/creatr-images/2019-08/f05a0150-bf9a-11e9-889d-4965fab2ec1e
https://i.ytimg.com/vi/UtVvSDrpOMc/maxresdefault.jpg
https://www.toptamilnews.com/static/c1e/client/88252/uploaded/7cea09b48120603052376715172beedd.gif
https://saravanamanian.files.wordpress.com/20

**categories**

In [ ]:
def pilgrimage(df):
  df.loc[df['categoryName'] == 'Hindu temple', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Buddhist temple', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Pagoda', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Church', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Catholic church', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Jain temple', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Cathedral', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Shrine', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Gurudwara', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Mosque', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Religious destination', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Bahá\'í house of worship', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Anglican church', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Monastery', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Meditation center', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Place of worship', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Pilgrimage place', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Catholic cathedral', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Basilica', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Cemetery', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Tomb', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Spiritist center', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Religious organization', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Protestant church', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Religious institution', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Christian church', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Baptist church', 'categoryName'] = 'Pilgrimage'
  df.loc[df['categoryName'] == 'Presbyterian church', 'categoryName'] = 'Pilgrimage'
  return df

In [ ]:
def hillstation(df):
  df.loc[df['categoryName'] == 'Hill station', 'categoryName'] = 'Hill station'
  df.loc[df['categoryName'] == 'Mountain peak', 'categoryName'] = 'Hill station'
  df.loc[df['categoryName'] == 'Mountain pass', 'categoryName'] = 'Hill station'
  df.loc[df['categoryName'] == 'Mountaineering class', 'categoryName'] = 'Hill station'
  df.loc[df['categoryName'] == 'Mountain cable car', 'categoryName'] = 'Hill station'  
  return df

In [ ]:
def wildlife(df):
  df.loc[df['categoryName'] == 'Bird watching area', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Aquarium', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Nature preserve', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Zoo', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'National park', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Wildlife and safari park', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Wildlife refuge', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Woods', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Wildlife park', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Animal protection organization', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Sanctuary', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'National reserve', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'Forestry service', 'categoryName'] = 'Wildlife'
  df.loc[df['categoryName'] == 'National forest', 'categoryName'] = 'Wildlife' 
  return df

In [ ]:
def adventure(df):
  df.loc[df['categoryName'] == 'Tourist attraction', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Garden', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Amusement park ride', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Water park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Scenic spot', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Memorial park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Rafting', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Lake', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Amusement park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Children\'s amusement center', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Theme park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Botanical garden', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Observatory', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'City park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Planetarium', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Campground', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Children\'s camp', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Reservoir', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Picnic ground', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Holiday park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Archaeological site', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Amusement center', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Ferry service', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Ferry terminal', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Technology park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Fountain', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Cricket ground', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Animal park', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Playground', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Fishing lake', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Canal', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Community garden', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Public golf course', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Park & ride', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Mountain cabin', 'categoryName'] = 'Adventure'
  df.loc[df['categoryName'] == 'Ecological park', 'categoryName'] = 'Adventure'

  return df  

In [ ]:
Historical_Places = [
'Fortress',
 'Historical place',
'Historical landmark',
'Castle',
'Art gallery',
'Monument',
'Cultural center',
'Ashram',
'Heritage building',
 'Heritage preservation',
'Sculpture',
'War memorial',
 'Military cemetery',
'Handicraft', 
'Cultural landmark']


def historicalplaces(df):
  for i in range(15):
    df.loc[df['categoryName'] == Historical_Places[i], 'categoryName'] = 'Historical_Places'
  return df

In [ ]:

Museum = [

'Handicraft museum',
'Historical place museum',
'Museum',
'History museum',
 'National museum',
 'Rail museum',
'Army museum',
'Local history museum',
'Archaeological museum',
'Natural history museum',
 'Technology museum',
'Maritime museum',
'Heritage museum',
"Children's museum",
'Art museum',
 'Open air museum',
'War museum',
'Science museum']


def museum(df):
  for i in range(len(Museum)):
    df.loc[df['categoryName'] == Museum[i], 'categoryName'] = 'Museum'
  return df

In [ ]:
Trekking = [

 'Hiking area',
'Bridge',
'Ravine', 
'Raft trip outfitter',
'Swimming lake',
'Waterfall',
'River', 
'Farmstay', 
'Fishing pond',
'Farm',
'Hot spring',
'Pond', 
'River port',
'Camping farm']


def trekking(df):
  for i in range(len(Trekking)):
    df.loc[df['categoryName'] == Trekking[i], 'categoryName'] = 'Trekking'
  return df

In [ ]:
Desert = ['Desert']


def desert(df):
  for i in range(len(Desert)):
    df.loc[df['categoryName'] == Desert[i], 'categoryName'] = 'Desert'
  return df

In [ ]:
Others = [

'Resort hotel',
'Movie studio',
'Market',
'Federal government office',
'Convention center',
 'Homestay',
'Manor house',
'Stadium',
'Andhra restaurant',
'Non-profit organization', 
'Travel agency',  'Indoor snowcenter', 
'Exhibition and trade centre', 
'Tour agency', 'Train station', 
'Performing arts theater', 
'Tour operator', 
'Tourist information center', 
       'Sightseeing tour agency',
      'Bus stop', 
       'Corporate office', 'Indoor lodging',        
       'Water damage restoration service', 
        'Event venue', 'Hotel', 
        'Training centre',      
        'Government office',       
        'Orphanage', 'Restaurant',
       'Vegetarian restaurant', 'South Indian restaurant', 'Food court',
       'Wedding venue', 'Hyderabadi restaurant', 'International airport',
       'Boat tour agency', 'Recreation center',       
       'Tea store', 'Photography studio', 'Observation deck',
        'Shopping mall',
       'Business center', 'Adventure sports center', 'Shooting range',
       'Fairground', 'Amphitheater', 
      'Shoe store', 'Spring', 'Store', 'Marina',
       'SCUBA tour agency', 'Sports complex', 
       'Saw mill', 
       'Cruise line company', 
       'Tourism development corporation', 'State Department of Tourism',
       'Auditorium', 'Cocktail bar', 'Cafe', 'Jewelry store',
       'Car rental agency', 'Lounge', 'Movie theater',
       'Punjabi restaurant', 'Night club', 'Bar', 'Bus station',
       'Toy store', 'Brewpub', 'Dharamashala',
       'Public university', 'Information services', 'Pizza restaurant',
       'Escape room center', 
       'University',  'Entertainment agency',
       'Adventure sports', 'Art school',  'Library',
       'School', 'Spiritist center', 'Mediation service', 'Boat club',
       'State government office', 'Water works',
       'Hydroelectric power plant', 'City Hall',
        'Taxi service',  'Hall',
       'Landmark', 'Information and referral services', 'Music store',
       'Cremation service', 'Coal supplier', 'Lodge',
       'Water',  'Biryani restaurant', 'Clothing store',
       'Community center', 'Hypermarket', 'Manufacturer',
       'Coaching center', 'Organic shop', 'Gym', 'Barbecue restaurant',
       'Grocery store', 'Parking lot', 'Electronics store',
       'Association or organization',  'Computer store',
       'Guest house', 'Polo club',
       'Fast food restaurant',
       'Korean restaurant', 
       'University department', 
       'Monument maker', 
       'Lodging', 'Men\'s clothing store',
       'Handicraft exporter', 'Provincial council',
       'Research institute', 'Family restaurant', 'Tea manufacturer',
       'Elevated', 'Boat rental service', 'Media company',
       'Houseboat rental service', 'Transportation service',
       'Cruise agency', 'Transportation infrastructure',
       'Seafood restaurant', 'Regional airport', 'Dive shop',
       'Supermarket', 'Inn', 'Cottage rental'
       ]




def others(df):
  for i in range(len(Others)):
    df.loc[df['categoryName'] == Others[i], 'categoryName'] = 'Others'
  return df

In [ ]:
def beach(df):
    df.loc[df['categoryName'] == 'Public beach', 'categoryName'] = 'Beach'
    df.loc[df['categoryName'] == 'Beach pavillion', 'categoryName'] = 'Beach'
    return df


In [ ]:
travel_data = pilgrimage(travel_data)
travel_data = hillstation(travel_data)
travel_data = wildlife(travel_data)
travel_data = adventure(travel_data)
travel_data = historicalplaces(travel_data)
travel_data = museum(travel_data)
travel_data = trekking(travel_data)
travel_data = desert(travel_data)
travel_data = others(travel_data)
travel_data = beach(travel_data)


In [ ]:
travel_data['categoryName'].value_counts()

Adventure            1347
Pilgrimage            580
Others                495
Historical_Places     302
Wildlife              214
Museum                175
Trekking               68
Beach                  37
Hill station           35
Island                 32
Desert                  1
Name: categoryName, dtype: int64

In [80]:
travel_data['imageUrls'].isnull().sum()

0

In [81]:
travel_data.to_excel('final_full_state_wise_data.xlsx', index=False)


In [82]:
cols = ['title','subTitle','imageUrls','categoryName','description','address','city','state','neighborhood',
   'reviewsTags', 'totalScore']

In [83]:
final_travel_data = pd.DataFrame(travel_data, columns=cols)


In [84]:
final_travel_data.shape

(3286, 11)

In [85]:
final_travel_data.to_excel('trained_state_wise_data.xlsx', index=False)

In [86]:
final_travel_data['title'].isnull().sum()

0